**INITIALIZATION**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**LIBRARIES AND DEPENDENCIES**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [3]:
#@ INSTALLING DEPENDENCIES: UNCOMMENT BELOW: 
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

In [4]:
#@ DOWNLOADING LIBRARIES AND DEPENDENCIES: 
from fastbook import *                                  # Getting all the Libraries. 
from fastai.callback.fp16 import *
from fastai.vision.all import *                         # Getting all the Libraries.

**GETTING THE DATASET**
- I will use **Imagenette** dataset here. 

In [5]:
#@ GETTING THE DATASET: 
path = untar_data(URLs.IMAGENETTE)                      # Path to the Dataset. 
path.ls()                                               # Inspecting the Dataset. 

(#3) [Path('/root/.fastai/data/imagenette2/train'),Path('/root/.fastai/data/imagenette2/val'),Path('/root/.fastai/data/imagenette2/noisy_imagenette.csv')]

**DATABLOCK AND DATALOADERS**
- I will get the dataset into **DataLoaders** object using the **Presizing**. 

In [7]:
#@ INITIALIZING DATABLOCK AND DATALOADERS:
dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),        # Initializing DataBlock and Category Block. 
                   get_items=get_image_files,                     # Getting Images. 
                   get_y=parent_label,                            # Getting Labels. 
                   item_tfms=Resize(460),                         # Resizing Images. 
                   batch_tfms=aug_transforms(size=224, 
                                             min_scale=0.75))     # Initializing DataBlock and Augmentation. 
dls = dblock.dataloaders(path, bs=64)                             # Initializing DataLoaders and Batchsize. 

**TRAINING THE MODEL**

In [9]:
#@ TRAINING THE MODEL: BASELINE:
model = xresnet50()                                               # Initializing the Model. 
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy).to_fp16()                       # Initializing the Learner. 
learn.fit_one_cycle(5, 3e-3)                                      # Training the Model. 

epoch,train_loss,valid_loss,accuracy,time
0,1.647606,2.069581,0.371546,02:18
1,1.243452,9.026949,0.259895,02:19
2,0.947631,0.984491,0.672890,02:18
3,0.738748,0.683733,0.783794,02:20
4,0.583327,0.576961,0.828603,02:19


**NORMALIZATION**
- When the model is training, it helps if the input data is normalized i.e has a mean of 0 and standard deviation of 1. 

In [10]:
#@ GETTING MEAN AND STANDARD DEVIATION: 
x, y = dls.one_batch()                          # Getting a Batch of Data. 
x.mean(dim=[0, 2, 3]), x.std(dim=[0, 2, 3])     # Getting Mean and SD. 

(TensorImage([0.4630, 0.4513, 0.4254], device='cuda:0'),
 TensorImage([0.2817, 0.2791, 0.3007], device='cuda:0'))

In [12]:
#@ INITIALIZING NORMALIZATION: 
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),                     # Initializing DataBlock and Category Block.
                       get_items=get_image_files,                              # Getting Images. 
                       get_y=parent_label,                                     # Getting Labels.
                       item_tfms=Resize(460),                                  # Resizing Images.
                       batch_tfms=[*aug_transforms(size=size, min_scale=0.75), # Initializing DataBlock.
                                   Normalize.from_stats(*imagenet_stats)])     # Normalization. 
    return dblock.dataloaders(path, bs=bs)                                     # Getting DataLoaders. 

#@ GETTING DATALOADERS: 
dls = get_dls(64, 224)                                            # Getting DataLoaders. 
#@ GETTING MEAN AND STANDARD DEVIATION: 
x, y = dls.one_batch()                                            # Getting a Batch of Data. 
x.mean(dim=[0, 2, 3]), x.std(dim=[0, 2, 3])                       # Getting Mean and SD. 

(TensorImage([-0.1507, -0.1438, -0.1446], device='cuda:0'),
 TensorImage([1.1244, 1.1514, 1.2135], device='cuda:0'))

In [13]:
#@ TRAINING THE MODEL: AFTER NORMALIZATION: 
model = xresnet50()                                               # Initializing the Model. 
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy).to_fp16()                       # Initializing the Learner. 
learn.fit_one_cycle(5, 3e-3)                                      # Training the Model. 

epoch,train_loss,valid_loss,accuracy,time
0,1.695970,2.884375,0.371173,02:16
1,1.254432,1.597718,0.537715,02:17
2,0.976779,1.007531,0.686333,02:16
3,0.717962,0.779845,0.743465,02:18
4,0.570493,0.562726,0.819642,02:16


**PROGRESSIVE RESIZING**
- Progressive Resizing is the process of gradually using larger and larger images as training progresses. 

In [15]:
#@ TRAINING THE MODEL: SMALL IMAGES: 
dls = get_dls(128, 128)                                           # Getting DataLoaders. 
learn = Learner(dls,xresnet50(),loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy).to_fp16()                       # Initializing Learner. 
learn.fit_one_cycle(4, 3e-3)                                      # Training the Model. 

epoch,train_loss,valid_loss,accuracy,time
0,1.841082,1.736076,0.485437,02:02
1,1.288558,1.820757,0.510456,02:02
2,0.961439,0.945185,0.700149,02:02
3,0.727146,0.690849,0.784914,02:02


In [ ]:
#@ TRAINING THE MODEL: LARGE IMAGES: 
learn.dls = get_dls(64, 224)                                      # Initializing DataLoaders. 
learn.fine_tune(5, 1e-3)                                          # Training the Model. 